In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !pip install --quiet scvi-colab
    from scvi_colab import install
    install()
    !pip install --quiet git+https://github.com/BayraktarLab/cell2location#egg=cell2location[tutorials]

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl

import cell2location
import scvi

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs

results_folder = './run_regression/result'

# create paths and names to results folders for reference regression and cell2location models
#ref_run_name = f'{results_folder}/reference_signatures'
#run_name = f'{results_folder}/cell2location_map'

import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/st_decon/cell2location/run_regression/stride_data1"

os.chdir(path)
os.listdir(path)

In [ ]:
import pandas as pd
#!pip install anndata
import anndata
import numpy as np
import itertools

# load simulated spatial data
# every column is a spot of sc
adata = pd.read_csv('adata.csv',index_col=0)

# read the true cell numbers
# every column is a spot of sc
obs = pd.read_csv('obs.csv',index_col=0)
obs.columns = adata.index

# read the true UMI count
#obs_2 = pd.read_csv(sp_data_folder + 'synthetic_ST_seed'+seed_numbers+'_1_umis.csv',index_col=0)
#obs_2.columns = ['nUMI_' + i  for i in obs_2.columns]

# create AnnData object
adata = anndata.AnnData(adata, obs=obs.T)
#adata.obs[obs_2.columns] = obs_2

# generate random positions
adata.obsm['X_spatial'] = np.array(list(itertools.product(np.arange(np.ceil(np.sqrt(adata.shape[0]))), 
                       np.arange(np.ceil(np.sqrt(adata.shape[0]))))))[0:adata.shape[0],:]

#design

adata_vis=adata

adata_vis.var['SYMBOL'] = adata_vis.var_names
#adata_vis.var.set_index('gene_ids', drop=True, inplace=True)

adata_vis.obs['Sample']='sample'

# find mitochondria-encoded (MT) genes
adata_vis.var['MT_gene'] = [gene.startswith('MT-') for gene in adata_vis.var['SYMBOL']]
    
# remove MT genes for spatial mapping (keeping their counts in the object)
#adata_vis.obsm['MT'] = adata_vis[:, adata_vis.var['MT_gene'].values].X.toarray()
#adata_vis = adata_vis[:, ~adata_vis.var['MT_gene'].values]

# load single cell reference used for simulation (validation set)
counts_validation = pd.read_csv('sc_count.csv',index_col=0)
#adata = pd.read_csv('adata.csv',index_col=0)
labels_validation = pd.read_csv('sc_lable.csv',index_col=0)
labels_validation.columns = counts_validation.index

labels_validation.index=['label']

adata_snrna_raw = anndata.AnnData(counts_validation, obs=labels_validation.T)

adata_ref=adata_snrna_raw

adata_ref.obs['Sample']='sample'

adata_ref.var['SYMBOL'] = adata_ref.var.index 
# rename 'GeneID-2' as necessary for your data
#adata_ref.var.set_index('GeneID-2', drop=True, inplace=True)

# delete unnecessary raw slot (to be removed in a future version of the tutorial)
del adata_ref.raw

from cell2location.utils.filtering import filter_genes
selected = filter_genes(adata_ref, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)

# filter the object
adata_ref = adata_ref[:, selected].copy()

import cell2location

adata_ref.obs['Method']='5'

adata_ref.obs_keys()

adata_ref.obs['Subset']=adata_ref.obs['label']

# prepare anndata for the regression model
cell2location.models.RegressionModel.setup_anndata(adata=adata_ref, 
                        # 10X reaction / sample / batch
                        batch_key='Sample', 
                        # cell type, covariate used for constructing signatures
                        labels_key='Subset', 
                        # multiplicative technical effects (platform, 3' vs 5', donor effect)
                        categorical_covariate_keys=['Method']
                       )

# create the regression model
from cell2location.models import RegressionModel
mod = RegressionModel(adata_ref) 

# view anndata_setup as a sanity check
mod.view_anndata_setup()

mod.train(max_epochs=250, use_gpu=True)

ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'

# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': True}
)

# Save model
mod.save(f"{ref_run_name}", overwrite=True)

#adata_ref.obs.tostring
# Save anndata object with results
adata_file = f"{ref_run_name}/sc.h5ad"
adata_ref.write(adata_file)
adata_file

adata_file = f"{ref_run_name}/sc.h5ad"
adata_ref = sc.read_h5ad(adata_file)
mod = cell2location.models.RegressionModel.load(f"{ref_run_name}", adata_ref)

# export estimated expression in each cluster
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}' 
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}' 
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver.iloc[0:5, 0:5]

adata_vis.obs["sample"]='sample'

# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
adata_vis = adata_vis[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

# prepare anndata for cell2location model
cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key="sample")

# create and train the model
mod = cell2location.models.Cell2location(
    adata_vis, cell_state_df=inf_aver, 
    # the expected average cell abundance: tissue-dependent 
    # hyper-prior which can be estimated from paired histology:
    N_cells_per_location=8,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection:
    detection_alpha=20
) 
mod.view_anndata_setup()

mod.train(max_epochs=30000, 
          # train using full data (batch_size=None)
          batch_size=None, 
          # use all data points in training because 
          # we need to estimate cell abundance at all locations
          train_size=1,
          use_gpu=True)

# plot ELBO loss history during training, removing first 100 epochs from the plot
mod.plot_history(1000)
plt.legend(labels=['full data training']);

# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_vis = mod.export_posterior(
    adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': True}
)

# Save model
mod.save(f"{run_name}", overwrite=True)

# mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

# Save anndata object with results
adata_file = f"{run_name}/sp.h5ad"
adata_vis.write(adata_file)
adata_file

adata_file = f"{run_name}/sp.h5ad"
adata_vis.write(adata_file)
adata_file

adata_file = f"{run_name}/ap1.csv"
adata_vis.obsm['means_cell_abundance_w_sf'].to_csv(adata_file)

